### Install necessary dependencies

In [1]:
!conda install -y BeautifulSoup4
!conda install -y lxml
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import requests

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB
    beautifulsoup4-4.8.0       |           py36_0         147 KB
    soupsieve-1.9.3            |           py36_0          60 KB
    openssl-1.1.1d             |       h7b6447c_3         3.7 MB
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.3-py36_0                 

The following packages will 

### Get the wikipedia table and store it in My_tabel

In [17]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'html5lib')
#print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'})
#print(My_table)
links = My_table.find_all('td')
#print(links)

### Fill Dataframe line  by line taking into account the requirements from coursera

In [18]:
#init Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns=column_names)

i=0
boroughNotAssigned = False

#loop through all entries in table
for link in links:
    
    #in case of postalcode
    if( i%3 == 0):
        postalcode = str(link).split('>')[1].split('<')[0]
        
    #in case of borough
    if ( i%3 == 1):
        
        if("Not assigned" in str(link)):
            boroughNotAssigned = True
            
        if("href" in str(link)):
            borough = str(link).split('>')[2]
            borough = borough.split("<")[0]
        else:
            borough = str(link).split(">")[1].split("<")[0]
    
    #in case of neighborhood       
    if ( i%3 == 2):
        
        link = (str(link)[:-6])
            
        if("href" in str(link)):
            neighborhood = str(link).split('>')[2]
            neighborhood = neighborhood.split("<")[0]
        else:
            neighborhood = str(link).split(">")[1].split("<")[0]
         
        #if neighborhood not assigned, give it borough
        if(neighborhood == "Not assigned"):
            neighborhood = borough
         
        # if borough assgined, add entry in Dataframe
        if(boroughNotAssigned == False):
            
            if(sum(neighborhoods["Postalcode"]==postalcode)):
                neighborhoods.replace((neighborhoods[neighborhoods["Postalcode"]==postalcode].Neighborhood).values,
                                       (neighborhoods[neighborhoods["Postalcode"]==postalcode].Neighborhood).values +', ' + neighborhood,inplace=True)
            else:
                neighborhoods = neighborhoods.append({'Postalcode': postalcode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood}, ignore_index=True)
            
        boroughNotAssigned = False
    i = i +1

In [19]:
neighborhoods.shape

(103, 3)